In [50]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import numpy as np
import pandas as pd
import importlib
import matplotlib.pyplot as plt
from pyarrow import table

import volpy_func_lib as vp
import volpy_func_ticker_lib as vtp
import load_clean_lib
import table_lib
import vol_strat_lib as vs
import option_returns as orpy

importlib.reload(vtp)
importlib.reload(vp)
importlib.reload(load_clean_lib)
importlib.reload(table_lib)
importlib.reload(vs)

import os

In [58]:
def Create_Security_map():
    base_dir      = load_clean_lib.Option_metrics_path_from_profile()
    bloomberg_dir = base_dir / "Tickers" / "index data"
    
    # Load Excel files
    bloomberg_df = pd.read_excel(bloomberg_dir / "bloomberg data name.xlsx")
    id_map_df = pd.read_excel(bloomberg_dir / "final map done_v2.xlsx")
    
    # Truncate the last two chars of ID_CUSIP and normalize
    bloomberg_df['cusip_key'] = (
        bloomberg_df['ID_CUSIP']
        .astype(str)
        .str[:-1]    # drop last two characters
        .str.strip()
        .str.upper()
    )
    
    # also normalize the id_map_df key
    id_map_df['cusip_key'] = (
        id_map_df['cusip']
        .astype(str)
        .str.strip()
        .str.upper()
    )
    
    # build the lookup (dropping any duplicate keys)
    name_map = (
        bloomberg_df
        .drop_duplicates(subset='cusip_key', keep='first')
        .set_index('cusip_key')['LONG_COMP_name']
    )
    
    # map into a new column 'name'
    id_map_df['name'] = id_map_df['cusip_key'].map(name_map)
    id_map_df['name'] = id_map_df['name'].str.replace('/The', '', regex=False)
    
    
    # apply lookup, defaulting to the original ticker if not in the dict
    id_map_df['ticker_out'] = (
        id_map_df['ticker']
        .map(vp.ticker_to_ticker_out)
        .fillna(id_map_df['ticker'])
    )
    
    # apply overrides, defaulting to the original name if ticker not in the dict
    id_map_df['name'] = (
        id_map_df['ticker']
        .map(vp.name_overrides)
        .fillna(id_map_df['name'])
    )
    id_map_df["Synthetic"] = False
    EWU_combined = {
        "ticker": "EWU_combined",
        "name": "United Kingdom ETF",
        "ticker_out": "EWU (full)",
        "cusip": "46435G334/46428669",
        "permno": "14907/83216",
        "secid": "106420",
        "cusip_key": "46435G334/46428669",
        "Synthetic": True,   
    }
    id_map_df = pd.concat([id_map_df, pd.DataFrame([EWU_combined])], ignore_index=True)
    id_map_df.to_excel(bloomberg_dir / "Security_map.xlsx", index=False)

In [12]:
# Paths
base_dir      = load_clean_lib.Option_metrics_path_from_profile()
bloomberg_dir = base_dir / "Tickers" / "index data"
input_dir     = base_dir / "Tickers" / "Input"

df = pd.read_csv( bloomberg_dir/"ticker_to_permno_lookup_fixed.csv")

In [15]:
import os
import pandas as pd

# 1. build list of tickers from your master df
tickers = df['ticker'].unique().tolist()

# 2. read-in each “returns and stock price.csv” and tag it with its ticker
all_returns = []
for t in tickers:
    path = os.path.join(input_dir, t, "returns and stock price.csv")
    if os.path.exists(path):
        tmp = pd.read_csv(path, dtype={'cusip': str})
        tmp['ticker'] = t
        all_returns.append(tmp)
    else:
        # warn if the file is missing
        print(f"Warning: no data for ticker {t} at {path}")

# 3. concatenate into one big DataFrame
returns_df = pd.concat(all_returns, ignore_index=True)

# 4. extract a permno→cusip mapping (assuming returns_df has a ‘permno’ column)
permno_to_cusip = (
    returns_df
    .drop_duplicates(subset='permno')[['permno', 'cusip']]
    .set_index('permno')['cusip']
)

# 5. map back onto your original df
df['cusip_1'] = df['permno_1'].map(permno_to_cusip)
df['cusip_2'] = df['permno_2'].map(permno_to_cusip)

df.to_csv(bloomberg_dir / "ticker_to_id_lookup.csv")

,dataset,ticker,permno_1,permno_2,Note,cusip_1,cusip_2
0,OEX,AA,24643,NaN,NaN,44320110,NaN
1,OEX,AAPL,14593,NaN,NaN,03783310,NaN
2,OEX,ABBV,13721,NaN,NaN,00287Y10,NaN
3,OEX,ABT,20482,NaN,NaN,00282410,NaN
4,OEX,ACN,89071,NaN,NaN,G1151C10,NaN
...,...,...,...,...,...,...,...
209,non OEX,USO,91208,NaN,NaN,91232N20,NaN
210,non OEX,UUP,91758,NaN,NaN,46141D20,NaN
211,non OEX,UVXY,13030,NaN,NaN,74347Y75,NaN
212,non OEX,VGK,91132,NaN,NaN,92204287,NaN


In [7]:
df

,dataset,ticker,permno_1,permno_2,Note
0,OEX,AA,24643,NaN,NaN
1,OEX,AAPL,14593,NaN,NaN
2,OEX,ABBV,13721,NaN,NaN
3,OEX,ABT,20482,NaN,NaN
4,OEX,ACN,89071,NaN,NaN
...,...,...,...,...,...
209,non OEX,USO,91208,NaN,NaN
210,non OEX,UUP,91758,NaN,NaN
211,non OEX,UVXY,13030,NaN,NaN
212,non OEX,VGK,91132,NaN,NaN


In [5]:
od_raw = vtp.concat_output_ticker_datasets(["TWX"], "od_raw")

In [13]:
od_raw["date"]

0         1996-01-04
1         1996-01-04
2         1996-01-04
3         1996-01-04
4         1996-01-04
             ...    
1564211   2018-06-14
1564212   2018-06-14
1564213   2018-06-14
1564214   2018-06-14
1564215   2018-06-14
Name: date, Length: 1564216, dtype: datetime64[ns]

In [12]:
od_raw[od_raw["date"].dt.year==2002]

,ticker,optionid,date,exdate,cp_flag,K,bid,ask,IV_om,volume,mid,n_trading_day,n_trading_day_exdate,t_days,t_TTM,c_days,c_TTM,F,IV,spread
299078,TWX,11825795,2002-01-02,2002-01-19,P,120.0,88.3,88.5,2.173500,0.0,88.40,1509,1521.0,12.0,0.047695,17,0.046575,31.627826,2.173500,0.2
299079,TWX,11825793,2002-01-02,2002-01-19,P,115.0,83.3,83.5,2.107030,0.0,83.40,1509,1521.0,12.0,0.047695,17,0.046575,31.627826,2.107030,0.2
299080,TWX,11825791,2002-01-02,2002-01-19,P,110.0,78.3,78.5,2.037669,0.0,78.40,1509,1521.0,12.0,0.047695,17,0.046575,31.627826,2.037669,0.2
299081,TWX,11825789,2002-01-02,2002-01-19,P,105.0,73.3,73.5,1.965141,0.0,73.40,1509,1521.0,12.0,0.047695,17,0.046575,31.627826,1.965141,0.2
299082,TWX,20428464,2002-01-02,2002-04-20,C,60.0,0.0,0.1,0.517134,0.0,0.05,1509,1583.0,74.0,0.294118,108,0.295890,31.784382,0.517134,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344109,TWX,20805939,2002-12-31,2003-04-19,P,25.0,11.8,12.0,0.526229,0.0,11.90,1760,1834.0,74.0,0.294118,109,0.298630,13.150584,0.526229,0.2
344110,TWX,20805941,2002-12-31,2003-04-19,P,35.0,21.8,22.0,NaN,0.0,21.90,1760,1834.0,74.0,0.294118,109,0.298630,13.150584,NaN,0.2
344111,TWX,21356262,2002-12-31,2003-02-22,P,27.5,14.3,14.5,0.649490,0.0,14.40,1760,1795.0,35.0,0.139110,53,0.145205,13.086603,0.649490,0.2
344112,TWX,21365980,2002-12-31,2003-02-22,P,42.5,29.3,29.5,NaN,0.0,29.40,1760,1795.0,35.0,0.139110,53,0.145205,13.086603,NaN,0.2


In [2]:
import pandas as pd
from pathlib import Path
import load_clean_lib

# Paths
base_dir      = load_clean_lib.Option_metrics_path_from_profile()
bloomberg_dir = base_dir / "Tickers" / "index data"
input_dir     = base_dir / "Tickers" / "Input"

# Load OEX constituents
oex_df = pd.read_csv(bloomberg_dir / "oex_constituents_long.csv")

# ——————————————————————————————————————————————————————————————
# 1) Build the OMX‐based lookup
# ——————————————————————————————————————————————————————————————
permno_records = []
for ticker in oex_df['ticker'].unique():
    fpath = input_dir / ticker / "returns and stock price.csv"
    if not fpath.exists():
        continue
    df = pd.read_csv(fpath, usecols=['date', 'permno', 'ticker'])
    permno_records.append(df)

permno_df = pd.concat(permno_records, ignore_index=True)

omx_with_permno = oex_df.merge(
    permno_df,
    on=['ticker', 'date'],
    how='left'
)

omx_ticker_lookup = (
    omx_with_permno
    .dropna(subset=['permno'])
    .groupby('ticker')['permno']
    .unique()
    .apply(list)
    .reset_index(name='permnos')
)

# Expand list‐column into separate permno_N columns
omx_expanded = pd.DataFrame(omx_ticker_lookup['permnos'].tolist())
omx_expanded.columns = [f'permno_{i+1}' for i in omx_expanded.columns]

omx_ticker_lookup = pd.concat(
    [omx_ticker_lookup[['ticker']].reset_index(drop=True),
     omx_expanded.reset_index(drop=True)],
    axis=1
)


# ——————————————————————————————————————————————————————————————
# 2) Build the “extra” lookup for tickers not in OMX (and not in vp.Index_tickers)
# ——————————————————————————————————————————————————————————————
existing       = set(omx_ticker_lookup['ticker'])
all_dirs       = [d.name for d in input_dir.iterdir() if d.is_dir()]
to_process     = [t for t in all_dirs
                  if t not in existing and t not in vp.Index_tickers]

extra_records = []
for ticker in to_process:
    fpath = input_dir / ticker / "returns and stock price.csv"
    if not fpath.exists():
        continue
    perms = (
        pd.read_csv(fpath, usecols=['permno'])
          ['permno']
          .dropna()
          .unique()
          .tolist()
    )
    extra_records.append({'ticker': ticker, 'permnos': perms})

ex_omx_ticker_lookup = pd.DataFrame(extra_records)

# Expand list‐column
ex_expanded = pd.DataFrame(ex_omx_ticker_lookup['permnos'].tolist())
ex_expanded.columns = [f'permno_{i+1}' for i in ex_expanded.columns]

ex_omx_ticker_lookup = pd.concat(
    [ex_omx_ticker_lookup[['ticker']].reset_index(drop=True),
     ex_expanded.reset_index(drop=True)],
    axis=1
)


# ——————————————————————————————————————————————————————————————
# 3) Merge both lookups and save only the combined table
# ——————————————————————————————————————————————————————————————
ticker_lookup_full = pd.concat(
    [omx_ticker_lookup, ex_omx_ticker_lookup],
    ignore_index=True
)

out_path = base_dir / "Tickers" / "ticker_to_permno_lookup.csv"
ticker_lookup_full.to_csv(out_path, index=False)

In [3]:
# Load Bloomberg data
bb_df = pd.read_csv(bloomberg_dir / "bloomberg data name.csv")

In [4]:
bb_df['ticker'] = bb_df['Ticker'].str.split().str[0]
bb_df.rename(columns={'ID_CUSIP': 'cusip'}, inplace=True)
bb_df = ticker_lookup_full.merge(bb_df, on='ticker', how='left').copy()
bb_df = bb_df.drop_duplicates(subset='ticker', keep='first')

In [5]:
bb_df

,ticker,permno_1,permno_2,kilde,Ticker,LONG_COMP_name,cusip,Issuer,Unnamed: 5,ID_CUSIP_NUM,Name,Unnamed: 8,Name 2
0,AA,24643.0,NaN,OEX,AA UN Equity,Alcoa Corp,013872106,US0138721065,47375937.0,013872106,Alcoa Corp,AA,ALCOA CORP
2,AAPL,14593.0,NaN,OEX,AAPL UW Equity,Apple Inc,037833100,US0378331005,101695.0,037833100,Apple Inc,AAPL,APPLE INC
5,ABBV,13721.0,NaN,OEX,ABBV UN Equity,AbbVie Inc,00287Y109,US00287Y1091,28046509.0,00287Y109,AbbVie Inc,ABBV,ABBVIE INC
6,ABT,20482.0,NaN,OEX,ABT UN Equity,Abbott Laboratories,002824100,US0028241000,100002.0,002824100,Abbott Laboratories,ABT,ABBOTT LABS
7,ACN,89071.0,NaN,OEX,ACN UN Equity,Accenture PLC,NaN,IE00B4BNMY34,300787.0,NaN,Accenture PLC,ACN,ACCENTURE PLC-A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,USO,46324.0,91208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,UUP,91758.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,UVXY,13030.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,VGK,91132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
bb_df[["ticker", "cusip", "LONG_COMP_name", "permno_1", "permno_2", "Issuer"]].to_csv(bloomberg_dir / "cusip_lookup.csv", index=False)

In [21]:
bb_df

,kilde,Ticker,LONG_COMP_name,ID_CUSIP,Issuer,Unnamed: 5,ID_CUSIP_NUM,Name,Unnamed: 8,Name 2
0,OEX,0203524D UN Equity,National Semiconductor Corp,637640103,US6376401039,101460,637640103,Texas Instruments Inc,#N/A Field Not Applicable,NATL SEMICONDUCT
1,OEX,1028411Q UN Equity,Gillette Co/The,375766102,US3757661026,101215,375766102,Procter & Gamble Co/The,#N/A Field Not Applicable,GILLETTE CO
2,OEX,1697067D UN Equity,Dow Chemical Co/The,260543103,US2605431038,48365107,260543103,Dow Inc,#N/A Field Not Applicable,DOW CHEMICAL CO
3,OEX,1715651D UN Equity,EIDP Inc,263534109,US2635341090,48365101,263534109,Corteva Inc,#N/A Field Not Applicable,EIDP INC
4,OEX,1837572D UN Equity,Nokia of America Corp,549463107,US5494631071,115790,549463107,Nokia Oyj,#N/A Field Not Applicable,NOKIA OF AMERICA
...,...,...,...,...,...,...,...,...,...,...
271,INDU,HON UQ Equity,Honeywell International Inc,438516106,US4385161066,100043,438516106,Honeywell International Inc,HON,HONEYWELL INTL
272,INDU,WBA UQ Equity,Walgreens Boots Alliance Inc,931427108,US9314271084,101617,931427108,Walgreens Boots Alliance Inc,WBA,WALGREENS BOOTS
273,INDU,AMZN UQ Equity,Amazon.com Inc,023135106,US0231351067,216952,023135106,Amazon.com Inc,AMZN,AMAZON.COM INC
274,INDU,NVDA UQ Equity,NVIDIA Corp,67066G104,US67066G1040,313657,67066G104,NVIDIA Corp,NVDA,NVIDIA CORP


In [20]:
ticker_lookup_full

,ticker,permno_1,permno_2
0,AA,24643.0,NaN
1,AAPL,14593.0,NaN
2,ABBV,13721.0,NaN
3,ABT,20482.0,NaN
4,ACN,89071.0,NaN
...,...,...,...
209,USO,46324.0,91208.0
210,UUP,91758.0,NaN
211,UVXY,13030.0,NaN
212,VGK,91132.0,NaN


In [34]:
ticker_permnos_df

,ticker,permnos
0,AA,[24643.0]
1,AAPL,[14593.0]
2,ABBV,[13721.0]
3,ABT,[20482.0]
4,ACN,[89071.0]
...,...,...
174,WMT,[55976.0]
175,WY,[39917.0]
176,WYE,[15667.0]
177,XOM,[11850.0]


In [20]:
import pandas as pd
import numpy as np
from pathlib import Path
import load_clean_lib

# --- 1) set up paths and load your oex_df ---
base_dir      = load_clean_lib.Option_metrics_path_from_profile()
bloomberg_dir = base_dir / "Tickers" / "index data"
input_dir     = base_dir / "Tickers" / "Input"

oex_con_path = bloomberg_dir / "oex_constituents_long.csv"
oex_df = pd.read_csv(oex_con_path)

# --- 2) preload each ticker’s date→permno mapping ---
permno_maps = {}
for ticker in oex_df['ticker'].unique():
    fpath = input_dir / ticker / "returns and stock price.csv"
    if not fpath.exists():
        continue
    df = (pd.read_csv(fpath)
            .loc[:, ['date', 'permno']])
    permno_maps[ticker] = df.set_index('date')['permno']

# --- 3) look up permno for each row in oex_df via .map() ---
def lookup_permno(row):
    mp = permno_maps.get(row['ticker'])
    if mp is None:
        return np.nan
    return mp.get(row['date'], np.nan)

oex_df['permno'] = oex_df.apply(lookup_permno, axis=1)

# now oex_df has a ‘permno’ column aligned by ticker + date

In [21]:
oex_df

,date,ticker,permno
0,1996-01-01,NSM,NaN
1,1996-01-01,G,NaN
2,1996-01-01,DOW,NaN
3,1996-01-01,DD,NaN
4,1996-01-01,1837572D,NaN
...,...,...,...
11755,2025-01-01,V,NaN
11756,2025-01-01,VZ,NaN
11757,2025-01-01,WFC,NaN
11758,2025-01-01,WMT,NaN


In [13]:
df_tmp = pd.read_csv(input_dir / "AAPL" / "returns and stock price.csv")
df_tmp

,permno,ticker,cusip,issuer,date,open,close,return
0,14593,AAPL,3783310,APPLE COMPUTER INC,1996-01-02,31.75000,32.12500,0.007843
1,14593,AAPL,3783310,APPLE COMPUTER INC,1996-01-03,31.87500,32.12500,0.000000
2,14593,AAPL,3783310,APPLE COMPUTER INC,1996-01-04,31.37500,31.56250,-0.017510
3,14593,AAPL,3783310,APPLE COMPUTER INC,1996-01-05,31.37500,34.25000,0.085149
4,14593,AAPL,3783310,APPLE COMPUTER INC,1996-01-08,34.00000,34.62500,0.010949
...,...,...,...,...,...,...,...,...
7295,14593,AAPL,3783310,APPLE INC,2024-12-24,255.28999,258.20001,0.011478
7296,14593,AAPL,3783310,APPLE INC,2024-12-26,257.63000,259.01999,0.003176
7297,14593,AAPL,3783310,APPLE INC,2024-12-27,253.06000,255.59000,-0.013242
7298,14593,AAPL,3783310,APPLE INC,2024-12-30,250.75000,252.20000,-0.013263


In [12]:
oex_df

,date,ticker,permno
0,1996-01-01,NSM,NaN
1,1996-01-01,G,NaN
2,1996-01-01,DOW,NaN
3,1996-01-01,DD,NaN
4,1996-01-01,1837572D,NaN
...,...,...,...
11755,2025-01-01,V,NaN
11756,2025-01-01,VZ,NaN
11757,2025-01-01,WFC,NaN
11758,2025-01-01,WMT,NaN
